# \_\_slots\_\_

- 파이썬은 인스턴스의 모든 속성을 dict에 동적 저장
- \_\_slots\_\_ 명령어를 통해 속성을 정적 정의 가능
- 장점: 속성 정의/호출 속도 향상 및 메모리 사용량 감소
- 단점: 정의 이후 속성 추가/제거 불가

참고:
1. https://www.christianbarra.com/posts/let-me-introduce-slots/
2. https://blog.usejournal.com/a-quick-dive-into-pythons-slots-72cdc2d334e7
3. https://thomas-cokelaer.info/tutorials/python/slots.html

## 사용 예시
\_\_init\_\_ 함수 전에 \_\_slots\_\_ = [] 형태로 정의

In [1]:
class MyClass(object):
    __slots__ = ['name', 'identifier']
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier
        self.set_up()

In [2]:
import ipython_memory_usage.ipython_memory_usage as imu
num = 1024*256
imu.start_watching_memory()

In [2] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 60.49 MiB


## 메모리 사용량 및 속도 비교

\_\_slots\_\_: MyClass1

In [3]:
class MyClass1(object):
    __slots__ = ['name', 'identifier']
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier

class MyClass2(object):
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier

In [3] used 0.0938 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 60.58 MiB


In [4]:
%timeit [MyClass1(1,1) for i in range(num)]
%timeit [MyClass2(1,1) for i in range(num)]

162 ms ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
214 ms ± 4.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
In [4] used 2.5039 MiB RAM in 14.99s, peaked 41.80 MiB above current, total RAM usage 63.09 MiB


In [5]:
x = [MyClass1(1,1) for i in range(num)]

In [5] used 15.6836 MiB RAM in 0.40s, peaked 0.00 MiB above current, total RAM usage 78.77 MiB


In [6]:
y = [MyClass2(1,1) for i in range(num)]

In [6] used 44.6758 MiB RAM in 0.44s, peaked 0.00 MiB above current, total RAM usage 123.45 MiB


### 사이즈 비교

In [7]:
from sys import getsizeof
print(getsizeof(x[0].__slots__), getsizeof(y[0].__dict__))

80 112
In [7] used -0.0078 MiB RAM in 0.10s, peaked 0.01 MiB above current, total RAM usage 123.44 MiB


## 속성을 추가하고 싶을 땐?

In [8]:
x[0].test_value = 'Some String'

AttributeError: 'MyClass1' object has no attribute 'test_value'

In [8] used 0.6094 MiB RAM in 0.22s, peaked 0.00 MiB above current, total RAM usage 124.05 MiB


In [5]:
class MyClass3(object):
    __slots__ = ['name', 'identifier', '__dict__']
    def __init__(self, name):
        self.name = name
        self.identifier = identifier

In [10]:
%timeit [MyClass3(1,1) for i in range(num)]
%timeit [MyClass2(1,1) for i in range(num)]

167 ms ± 2.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
213 ms ± 2.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
In [10] used -0.8867 MiB RAM in 15.40s, peaked 42.64 MiB above current, total RAM usage 123.18 MiB


In [11]:
z = [MyClass3(1,1) for i in range(num)]

In [11] used 18.9727 MiB RAM in 0.46s, peaked 0.00 MiB above current, total RAM usage 142.15 MiB


In [12]:
a = [MyClass2(1,1) for i in range(num)]

In [12] used 44.6992 MiB RAM in 0.43s, peaked 0.00 MiB above current, total RAM usage 186.85 MiB


In [13]:
a[0].test_value = 'Some String'
z[0].test_value = 'Some String'
print(z[0].name, z[0].identifier, z[0].__dict__, z[0].test_value)

1 1 {'test_value': 'Some String'} Some String
In [13] used -1.5156 MiB RAM in 0.10s, peaked 1.52 MiB above current, total RAM usage 185.34 MiB


In [14]:
print(getsizeof(z[0].__slots__), getsizeof(z[0].__dict__), getsizeof(a[0].__dict__))

88 112 112
In [14] used 0.0000 MiB RAM in 0.11s, peaked 0.00 MiB above current, total RAM usage 185.34 MiB
